 - https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [14]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
#lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
print(lookup_tensor)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([0])
tensor([[ 1.8782, -0.5666,  0.4016, -0.1153,  0.3170]],
       grad_fn=<EmbeddingBackward>)


## N-Gram

In [ ]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [16]:
test_sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

In [15]:
# 입력 데이터 토큰화, 
# ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [21]:
vocab = set(test_sentence)
print(vocab)

{"deserv'd", 'shame,', 'now,', 'trenches', 'it', 'praise.', 'where', 'in', 'shall', "excuse,'", 'to', 'own', 'weed', "'This", 'see', 'so', 'thou', 'much', 'Shall', 'count,', 'winters', 'my', 'all', 'How', "youth's", 'answer', 'gazed', 'livery', 'Were', 'thine', 'Will', "beauty's", 'held:', 'warm', 'This', 'proud', 'couldst', 'eyes,', 'Then', 'be', 'Proving', 'his', 'old', 'lusty', 'days;', 'thriftless', 'a', 'Where', 'dig', "totter'd", 'When', 'made', 'besiege', 'sunken', 'when', 'brow,', 'more', 'field,', 'make', 'of', 'Thy', 'praise', 'beauty', 'treasure', 'fair', 'and', 'old,', 'blood', 'lies,', 'new', 'child', 'succession', 'sum', 'small', 'asked,', 'within', "feel'st", 'by', 'To', 'forty', 'cold.', 'mine', 'thine!', 'deep', 'an', 'And', 'worth', 'the', 'being', 'say,', 'thy', 'use,', 'art', 'were', 'If', 'all-eating', 'on'}


In [25]:
word_to_ix = {word : i for i, word in enumerate(vocab)}

In [27]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [28]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [31]:
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. model을 통과할 입력 데이터 준비 
        # (단어를 정수로 변환하고 그것들을 tensor에 감싼다)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        print(context_idxs)
        # Step 2. 새로운 instance를 통과시키기 전에 축적된 gradients를 터치에 Recall 
        # 이전 instance로 gradients를 Zero out
        model.zero_grad()

        # Step 3. Model에 forward
        log_probs = model(context_idxs)

        # Step 4. loss function 적용. (loss_function(y_pred, y))
        # y도 tensor에 정수화해서 넣어줌
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. backward에 넣고 gredient 업데이트
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

tensor([50, 79])
tensor([79, 20])
tensor([20,  8])
tensor([ 8, 52])
tensor([52, 90])
tensor([90, 55])
tensor([55, 85])
tensor([85, 48])
tensor([48, 83])
tensor([83,  3])
tensor([3, 7])
tensor([ 7, 90])
tensor([90, 31])
tensor([31, 57])
tensor([57, 60])
tensor([60, 24])
tensor([24, 35])
tensor([35, 27])
tensor([27, 15])
tensor([15, 26])
tensor([26, 96])
tensor([96,  2])
tensor([ 2, 30])
tensor([30, 39])
tensor([39, 46])
tensor([46, 49])
tensor([49, 12])
tensor([12, 59])
tensor([59, 73])
tensor([73, 86])
tensor([86, 32])
tensor([32, 38])
tensor([38, 88])
tensor([88, 74])
tensor([74,  6])
tensor([ 6, 22])
tensor([22, 90])
tensor([90, 62])
tensor([62, 68])
tensor([68, 47])
tensor([47, 22])
tensor([22, 87])
tensor([87, 63])
tensor([63, 59])
tensor([59, 90])
tensor([90, 43])
tensor([43, 44])
tensor([44, 78])
tensor([78, 89])
tensor([89, 75])
tensor([75, 29])
tensor([29, 11])
tensor([11, 83])
tensor([83, 53])
tensor([53, 37])
tensor([37, 28])
tensor([28, 84])
tensor([84, 95])
tensor([95,  1])

tensor([28, 84])
tensor([84, 95])
tensor([95,  1])
tensor([ 1, 65])
tensor([65, 45])
tensor([45,  5])
tensor([ 5, 23])
tensor([23, 17])
tensor([17, 56])
tensor([56, 61])
tensor([61,  0])
tensor([ 0, 90])
tensor([90, 31])
tensor([31, 91])
tensor([91, 94])
tensor([94, 16])
tensor([16, 36])
tensor([36, 25])
tensor([25, 13])
tensor([13, 64])
tensor([64, 70])
tensor([70, 59])
tensor([59, 81])
tensor([81, 18])
tensor([18, 72])
tensor([72, 21])
tensor([21, 19])
tensor([19, 65])
tensor([65, 58])
tensor([58, 21])
tensor([21, 42])
tensor([42,  9])
tensor([ 9, 40])
tensor([40, 41])
tensor([41, 62])
tensor([62, 77])
tensor([77, 71])
tensor([71, 82])
tensor([82, 34])
tensor([34, 93])
tensor([93, 10])
tensor([10, 39])
tensor([39, 69])
tensor([69, 51])
tensor([51, 54])
tensor([54, 16])
tensor([16, 92])
tensor([92, 66])
tensor([66, 85])
tensor([85, 14])
tensor([14, 90])
tensor([90, 67])
tensor([67, 33])
tensor([33, 54])
tensor([54, 16])
tensor([16, 76])
tensor([76,  4])
tensor([50, 79])
tensor([79, 20

In [35]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    print(context)
    target = raw_text[i]
    print(target)
    data.append((context, target))
print(data[:5])

['We', 'are', 'to', 'study']
about
['are', 'about', 'study', 'the']
to
['about', 'to', 'the', 'idea']
study
['to', 'study', 'idea', 'of']
the
['study', 'the', 'of', 'a']
idea
['the', 'idea', 'a', 'computational']
of
['idea', 'of', 'computational', 'process.']
a
['of', 'a', 'process.', 'Computational']
computational
['a', 'computational', 'Computational', 'processes']
process.
['computational', 'process.', 'processes', 'are']
Computational
['process.', 'Computational', 'are', 'abstract']
processes
['Computational', 'processes', 'abstract', 'beings']
are
['processes', 'are', 'beings', 'that']
abstract
['are', 'abstract', 'that', 'inhabit']
beings
['abstract', 'beings', 'inhabit', 'computers.']
that
['beings', 'that', 'computers.', 'As']
inhabit
['that', 'inhabit', 'As', 'they']
computers.
['inhabit', 'computers.', 'they', 'evolve,']
As
['computers.', 'As', 'evolve,', 'processes']
they
['As', 'they', 'processes', 'manipulate']
evolve,
['they', 'evolve,', 'manipulate', 'other']
processes
[

In [36]:
class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix)  # example

tensor([ 2, 45, 25, 40])